In [1]:
import os

In [2]:
%pwd

'f:\\zizo\\MLOPS\\datascienceproject\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\zizo\\MLOPS\\datascienceproject'

In [5]:
import pandas as pd

data = pd.read_csv("artifacts/data_ingestion/Temp_dataset.csv")
data.head()

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,TAVG,TAVG_ATTRIBUTES
0,EGM00062318,"ALEXANDRIA INTERNATIONAL, EG",31.184,29.949,-1.8,1/1/1957,15.7,"H,,S"
1,EGM00062318,"ALEXANDRIA INTERNATIONAL, EG",31.184,29.949,-1.8,1/4/1957,11.1,"H,,S"
2,EGM00062318,"ALEXANDRIA INTERNATIONAL, EG",31.184,29.949,-1.8,1/6/1957,11.7,"H,,S"
3,EGM00062318,"ALEXANDRIA INTERNATIONAL, EG",31.184,29.949,-1.8,1/7/1957,12.6,"H,,S"
4,EGM00062318,"ALEXANDRIA INTERNATIONAL, EG",31.184,29.949,-1.8,1/8/1957,10.4,"H,,S"


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21198 entries, 0 to 21197
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STATION          21198 non-null  object 
 1   NAME             21198 non-null  object 
 2   LATITUDE         21198 non-null  float64
 3   LONGITUDE        21198 non-null  float64
 4   ELEVATION        21198 non-null  float64
 5   DATE             21198 non-null  object 
 6   TAVG             21198 non-null  float64
 7   TAVG_ATTRIBUTES  21198 non-null  object 
dtypes: float64(4), object(4)
memory usage: 1.3+ MB


In [7]:
data.isnull().sum()

STATION            0
NAME               0
LATITUDE           0
LONGITUDE          0
ELEVATION          0
DATE               0
TAVG               0
TAVG_ATTRIBUTES    0
dtype: int64

In [8]:
data.shape

(21198, 8)

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [10]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema =schema,
        )
        return data_validation_config



In [11]:
import os
from src.datascience import logger

In [12]:
class DataValidation:
    def __init__(self,config: DataValidationConfig):
        self.config = config
        
    def validate_all_columns(self)-> bool:
        try: 
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            all_schema = self.config.all_schema.keys()
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"Validation status: {validation_status}")
            return validation_status
        except Exception as e:
            raise e


In [14]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config = data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-03-11 03:58:36,761: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-11 03:58:36,764: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-11 03:58:36,768: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-11 03:58:36,770: INFO: common: created directory at: artifacts]
[2025-03-11 03:58:36,771: INFO: common: created directory at: artifacts/data_validation]
